# $K$-means clustering 

In this chapter, we will introduce the $K$-means clustering algorithm. 

Watch the 8-minute video below for a visual explanation of $K$-means clustering.

```{admonition} Video

<iframe width="700" height="394" src="https://www.youtube.com/embed/4b5d3muPQmA?start=14" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining $K$-Means Clustering, by StatQuest](https://www.youtube.com/embed/4b5d3muPQmA?start=14)

```


## Clustering

_Clustering_ refers to a very broad set of techniques for finding subgroups, or clusters, in a data set. When we cluster the observations of a data set, we seek to partition them into distinct groups so that the observations within each group are quite similar to each other, while observations in different groups are quite different from each other. Of course, to make this concrete, we must define what it means for two or more observations to be similar or different. Indeed, this is often a domain-specific consideration that must be made based on knowledge of the data being studied. 

For instance, suppose that we have a set of n observations, each with $D$ features. We might define the similarity between two observations as the Euclidean distance between them, which is given by features. The n observations could correspond to tissue samples for patients with breast cancer, and the $D$ features could correspond to measurements collected for each tissue sample; these could be clinical measurements, such as tumour stage or grade, or they could be gene expression measurements. We may have a reason to believe that there is some heterogeneity among the n tissue samples; for instance, perhaps there are a few different unknown subtypes of breast cancer. Clustering could be used to find these subgroups. This is an unsupervised problem because we are trying to discover structure—in this case, distinct clusters—on the basis of a data set. The goal in supervised problems, on the other hand, is to try to predict some outcome vector such as survival time or response to drug treatment.

Both clustering and PCA seek to simplify the data via a small number of summaries, but their mechanisms are different:

- PCA looks to find a low-dimensional representation of the observations that explain a good fraction of the variance;
- Clustering looks to find homogeneous subgroups among the observations.

Another application of clustering arises in marketing. We may have access to a large number of measurements (e.g. median household income, occupation, distance from nearest urban area, and so forth) for a large number of people. Our goal is to perform market segmentation by identifying subgroups of people who might be more receptive to a particular form of advertising, or more likely to purchase a particular product. The task of performing market segmentation amounts to clustering the people in the data set.

Since clustering is popular in many fields, there exist a great number of clustering methods. In this section we focus on perhaps the two best-known clustering approaches: $K$-_means clustering_ and _hierarchical clustering_. In $K$-means clustering, we seek to partition the observations into a pre-specified number of clusters. On the other hand, in hierarchical clustering, we do not know in advance how many clusters we want; in fact, we end up with a tree-like visual representation of the observations, called a dendrogram, that allows us to view at once the clusterings obtained for each possible number of clusters, from 1 to n. There are advantages and disadvantages to each of these clustering approaches, which we highlight in this chapter.

In general, we can cluster observations on the basis of the features in order to identify subgroups among the observations, or we can cluster features on the basis of the observations in order to discover subgroups among the features. In what follows, for simplicity we will discuss clustering observations on the basis of the features, though the converse can be performed by simply transposing the data matrix.

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import svd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy

%matplotlib inline

## $K$-means clustering

$K$-means clustering is a simple and elegant approach for partitioning a data set into K distinct, non-overlapping clusters. To perform $K$-means clustering, we must first specify the desired number of clusters $K$; then the $K$-means algorithm will assign each observation to exactly one of the $K$ clusters. Figure 12.7 shows the results obtained from performing $K$-means
clustering on a simulated example consisting of 150 observations in two dimensions, using three different values of K.

The $K$-means clustering procedure results from a simple and intuitive mathematical problem. We begin by defining some notation. Let $C_1, \ldots, C_K$ denote sets containing the indices of the observations in each cluster. These sets satisfy two properties:

1. $C_1 \cup C_2 \cup \ldots \cup C_K = {1, \ldots , N} $. In other words, each observation belongs to at least one of the $K$ clusters.
2. $C_k \cap C_{k′} = \Phi \text{ for all } k \neq k′$. In other words, the clusters are non-overlapping: no observation belongs to more than one cluster.

For instance, if the ith observation is in the $k\text{th}$ cluster, then $i \in C_k $. The idea behind $K$-means clustering is that a good clustering is one for which the within-cluster variation is as small as possible. The within-cluster variation
for cluster $C_k$ is a measure $W(C_k)$ of the amount by which the observations within a cluster differ from each other. Hence we want to solve the problem

```{math}
:label: kmeans-problem
\min_{C_1, \ldots, C_K} \sum_{k=1}^K W(C_k).
```

In words, this formula says that we want to partition the observations into $K$ clusters such that the total within-cluster variation, summed over all $K$ clusters, is as small as possible.

In order to make solving Equation {eq}`kmeans-problem` actionable we need to define the within-cluster variation. There are many possible ways to define this concept, but by far the most common choice involves _squared Euclidean distance_. That is, we define

```{math}
:label: within-cluster-var
W(C_k) = \frac{1}{|C_k|} \sum_{i,j \in C_k} \sum_{d=1}^{D} \left( x_{i,d} - x_{j,d} \right)^2,
```

where $|C_k|$ denotes the number of observations in the $k\text{th}$  cluster. In other words, the within-cluster variation for the kth cluster is the sum of all of the pairwise squared Euclidean distances between the observations in the $k\text{th}$ cluster, divided by the total number of observations in the $k\text{th}$  cluster.

Combining Equations {eq}`kmeans-problem` and {eq}`within-cluster-var`, gives the optimization problem that defines
K-means clustering,

```{math}
:label: kmeans-obj
\min_{C_1, \ldots, C_K} \left\{\sum_{k=1}^K \frac{1}{|C_k|} \sum_{i,j \in C_k} \sum_{d=1}^{D} \left( x_{i,d} - x_{j,d} \right)^2\right\}.
```

Now, we would like to find an algorithm to solve Equation {eq}`kmeans-obj`—that is, a method to partition the observations into K clusters such that the objective of Equation {eq}`kmeans-obj` is minimised. This is in fact a very difficult problem to solve precisely, since there are almost $K^N$ ways to partition $N$ observations into $K$ clusters. This is a huge number unless $K$ and $N$ are tiny! Fortunately, a very simple algorithm can be shown to provide a local optimum—a pretty good
solution—to the $K$-means optimisation problem {eq}`kmeans-obj`. This approach is laid out in {prf:ref}`alg:kmeans-algorithm`.


```{prf:algorithm} K-Means Clustering
:label: alg:kmeans-algorithm

- Randomly assign a number, from $1$ to $K$, to each of the observations. These serve as initial cluster assignments for the observations.

- Iterate until the cluster assignments stop changing:

    - For each of the $K$ clusters, compute the cluster centroid, which is the the vector of the $D$ feature means of the observations in the $k\text{th}$ cluster.

    - Assign each observation to the cluster for which the squared Euclidean distance between the observation and the cluster centroid is smallest.
```


In [ ]:
# generate data
np.random.seed(21)
X = np.random.standard_normal((50, 2))
X[:25, 0] = X[:25, 0] + 3
X[:25, 1] = X[:25, 1] - 4

In [ ]:
n_clusters = 2
km1 = KMeans(n_clusters=n_clusters, n_init=20)
km1.fit(X)

n_clusters = 3
km2 = KMeans(n_clusters=n_clusters, n_init=20)
km2.fit(X)

In [ ]:
print(km1.labels_)
print(dir(km1))  # we can use dir to see other saved attributes

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.scatter(X[:, 0], X[:, 1], s=40, c=km1.labels_, cmap=plt.cm.prism)
ax1.set_title("$K$-means Clustering Results with K=2")
ax1.scatter(
    km1.cluster_centers_[:, 0],
    km1.cluster_centers_[:, 1],
    marker="+",
    s=100,
    c="k",
    linewidth=2,
)

ax2.scatter(X[:, 0], X[:, 1], s=40, c=km2.labels_, cmap=plt.cm.prism)
ax2.set_title("$K$-means Clustering Results with K=3")
ax2.scatter(
    km2.cluster_centers_[:, 0],
    km2.cluster_centers_[:, 1],
    marker="+",
    s=100,
    c="k",
    linewidth=2,
)
plt.show()

Hierarchical Clustering

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 18))

for linkage, cluster, ax in zip(
    [hierarchy.complete(X), hierarchy.average(X), hierarchy.single(X)],
    ["c1", "c2", "c3"],
    [ax1, ax2, ax3],
):
    cluster = hierarchy.dendrogram(linkage, ax=ax, color_threshold=0)

ax1.set_title("Complete Linkage")
ax2.set_title("Average Linkage")
ax3.set_title("Single Linkage")
plt.show()

Clustering the Observations of the NCI60 Data

In [ ]:
X = pd.read_csv("https://github.com/pykale/transparentML/raw/main/data/NCI60_data.csv")
y = pd.read_csv("https://github.com/pykale/transparentML/raw/main/data/NCI60_labs.csv")

In [ ]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), index=y.iloc[:, 0], columns=X.columns)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 20))

for linkage, cluster, ax in zip(
    [hierarchy.complete(X_scaled), hierarchy.average(X), hierarchy.single(X_scaled)],
    ["c1", "c2", "c3"],
    [ax1, ax2, ax3],
):
    cluster = hierarchy.dendrogram(
        linkage,
        labels=X_scaled.index,
        orientation="right",
        color_threshold=0,
        leaf_font_size=10,
        ax=ax,
    )

ax1.set_title("Complete Linkage")
ax2.set_title("Average Linkage")
ax3.set_title("Single Linkage")
plt.show()

In [ ]:
plt.figure(figsize=(10, 20))
cut4 = hierarchy.dendrogram(
    hierarchy.complete(X_scaled),
    labels=X_scaled.index,
    orientation="right",
    color_threshold=140,
    leaf_font_size=10,
)
plt.vlines(
    140, 0, plt.gca().yaxis.get_data_interval()[1], colors="r", linestyles="dashed"
)
plt.show()

$K$-means

In [ ]:
np.random.seed(21)
km3 = KMeans(n_clusters=4, n_init=50)
km3.fit(X_scaled)

In [ ]:
km3.labels_

Combine with PCA

In [ ]:
pca = PCA()
X_scaled_ = StandardScaler().fit_transform(X)
df_plot = pd.DataFrame(pca.fit_transform(X_scaled_))

In [ ]:
plt.figure(figsize=(10, 20))
pca_cluster = hierarchy.dendrogram(
    hierarchy.complete(X_scaled),
    labels=X_scaled.index,
    orientation="right",
    color_threshold=100,
    leaf_font_size=10,
)

In [ ]:
# Hierarchy based on Principal Components 1 to 5
plt.figure(figsize=(10, 20))
pca_cluster = hierarchy.dendrogram(
    hierarchy.complete(df_plot.iloc[:, :5]),
    labels=X_scaled.index,
    orientation="right",
    color_threshold=100,
    leaf_font_size=10,
)

## Exercises

min 3 max 5

